# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,549.095473,-0.217332,1234,548.878141,549.312806,1097.756282,1098.625611
6,716.475530,-0.219229,1234,716.256301,716.694759,1432.512603,1433.389517
7,158.881960,-0.039973,1234,158.841987,158.921933,317.683974,317.843866
8,676.978836,0.895253,1234,677.874089,676.083583,1355.748177,1352.167166
9,554.499209,0.534699,1234,555.033908,553.964510,1110.067816,1107.929020
10,893.695277,-0.995186,1234,892.700091,894.690463,1785.400183,1789.380927
11,670.040718,0.833889,1234,670.874607,669.206829,1341.749213,1338.413657
12,920.498121,0.884191,1234,921.382312,919.613930,1842.764625,1839.227860
13,537.076742,-0.011261,1234,537.065481,537.088004,1074.130963,1074.176007
14,235.145557,0.974386,1234,236.119943,234.171171,472.239887,468.342342


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-548.878141,549.312806
6,-716.256301,716.694759
7,-158.841987,158.921933
8,-677.874089,676.083583
9,-555.033908,553.964510
10,-892.700091,894.690463
11,-670.874607,669.206829
12,-921.382312,919.613930
13,-537.065481,537.088004
14,-236.119943,234.171171


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-548.878141,549.312806
6,-716.256301,716.694759
7,-158.841987,158.921933
8,-677.874089,676.083583
9,-555.033908,553.964510
10,-892.700091,894.690463
11,-670.874607,669.206829
12,-921.382312,919.613930
13,-537.065481,537.088004
14,-236.119943,234.171171


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-548.87814124,  549.31280571],
       [-716.25630145,  716.69475872],
       [-158.8419868 ,  158.92193322],
       [-677.87408866,  676.08358311],
       [-555.03390816,  553.96450996],
       [-892.70009145,  894.69046334],
       [-670.87460653,  669.20682867],
       [-921.3823123 ,  919.61393019],
       [-537.06548129,  537.08800357],
       [-236.11994349,  234.17117109]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1095.756282,-1098.625611
6,1430.512603,-1433.389517
7,315.683974,-317.843866
8,1353.748177,-1352.167166
9,1108.067816,-1107.929020
10,1783.400183,-1789.380927
11,1339.749213,-1338.413657
12,1840.764625,-1839.227860
13,1072.130963,-1074.176007
14,470.239887,-468.342342


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.549095,0.391334
6,0.716476,0.390386
7,0.158882,0.480013
8,0.676979,0.947626
9,0.554499,0.767350
10,0.893695,0.002407
11,0.670041,0.916944
12,0.920498,0.942096
13,0.537077,0.494369
14,0.235146,0.987193


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,549.095473,-0.217332
6,716.475530,-0.219229
7,158.881960,-0.039973
8,676.978836,0.895253
9,554.499209,0.534699
10,893.695277,-0.995186
11,670.040718,0.833889
12,920.498121,0.884191
13,537.076742,-0.011261
14,235.145557,0.974386


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
